# QUESTIONS

1. Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: 
   
   1. `playerID`
   2. `schoolID`
   3. `salary`
   4. `yearID`
   5. `teamID` *associated with each salary*
   
   Order the table by salary (highest to lowest) and print out the table in your report.

2. This three-part question requires you to calculate batting average (number of
   hits divided by the number of at-bats)

    - Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report.
    - Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results.
    - Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results.

3. Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Plotly Express to visualize the comparison. What do you learn?


In [22]:
import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px

# careful to list your path to the file or save it in the same place as your .qmd or .py file
sqlite_file = "lahmansbaseballdb.sqlite"
con = sqlite3.connect(sqlite_file)

q = "SELECT * FROM allstarfull LIMIT 5"
results = pd.read_sql_query(q, con)

results

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5


In [23]:
# This is our master table
q = """--sql
    SELECT * 
    FROM sqlite_master 
    WHERE type='table'
    """
table = pd.read_sql_query(q, con)
table_names = table.filter(["name"])["name"].to_list()
table_names

['allstarfull',
 'appearances',
 'awardsmanagers',
 'awardsplayers',
 'awardssharemanagers',
 'awardsshareplayers',
 'batting',
 'battingpost',
 'collegeplaying',
 'divisions',
 'fielding',
 'fieldingof',
 'fieldingofsplit',
 'fieldingpost',
 'halloffame',
 'homegames',
 'leagues',
 'managers',
 'managershalf',
 'parks',
 'people',
 'pitching',
 'pitchingpost',
 'salaries',
 'schools',
 'seriespost',
 'teams',
 'teamsfranchises',
 'teamshalf']

In [24]:
q = """--sql
    SELECT * FROM schools
    WHERE name_full = "Brigham Young University-Idaho"
    """

schools = pd.read_sql_query(q, con)
byui = schools.get("schoolID")[0]
schools

,schoolID,name_full,city,state,country
0,idbyuid,Brigham Young University-Idaho,Rexburg,ID,USA


In [35]:
q = """--sql
    SELECT people.nameGiven AS `Given Name`,
           people.playerID AS `Player ID`,
           collegeplaying.schoolID AS `School ID`,
           salaries.teamID AS `Team ID`,
           salaries.yearID AS `Year`,
           salaries.salary AS `Salary`
    FROM people
    JOIN collegeplaying ON people.playerID = collegeplaying.playerID
    JOIN salaries ON people.playerID = salaries.playerID
    WHERE schoolID = 'idbyuid'
    ORDER BY salary DESC
    """

player_info = pd.read_sql_query(q, con)
player_info = player_info.drop_duplicates().reset_index(drop=True)
player_info = player_info.replace("idbyuid", "Brigham Young University-Idaho")
player_info["Salary"] = player_info["Salary"].map(lambda x: f"$ {x:,.0f}")

player_info

,Given Name,Player ID,School ID,Team ID,Year,Salary
0,Matthew Raymond,lindsma01,Brigham Young University-Idaho,CHA,2014,"$ 4,000,000"
1,Matthew Raymond,lindsma01,Brigham Young University-Idaho,BAL,2012,"$ 3,600,000"
2,Matthew Raymond,lindsma01,Brigham Young University-Idaho,COL,2011,"$ 2,800,000"
3,Matthew Raymond,lindsma01,Brigham Young University-Idaho,CHA,2013,"$ 2,300,000"
4,Matthew Raymond,lindsma01,Brigham Young University-Idaho,HOU,2010,"$ 1,625,000"
5,Garrett Charles,stephga01,Brigham Young University-Idaho,SLN,2001,"$ 1,025,000"
6,Garrett Charles,stephga01,Brigham Young University-Idaho,SLN,2002,"$ 900,000"
7,Garrett Charles,stephga01,Brigham Young University-Idaho,SLN,2003,"$ 800,000"
8,Garrett Charles,stephga01,Brigham Young University-Idaho,SLN,2000,"$ 550,000"
9,Matthew Raymond,lindsma01,Brigham Young University-Idaho,FLO,2009,"$ 410,000"


In [26]:
q = """--sql
    SELECT playerID, yearID, H AS hits, AB AS at_bat
    FROM batting
    WHERE AB > 0
    """

batting_avgs = pd.read_sql_query(q, con)
batters = (
    batting_avgs.groupby("playerID")
    .agg(at_bat=("at_bat", "sum"), hits=("hits", "sum"))
    .assign(batting_average=lambda x: x.hits / x.at_bat)
)
batters.sort_values("batting_average", ascending=False).head().reset_index()

,playerID,at_bat,hits,batting_average
0,lawsost01,1,1,1.0
1,stoneti01,1,1,1.0
2,alanirj01,1,1,1.0
3,alberan01,1,1,1.0
4,moharjo01,1,1,1.0


In [27]:
q = """--sql
    SELECT playerID, yearID, H AS hits, AB AS at_bat
    FROM batting
    WHERE AB >= 100
    """

high_batting_avgs = pd.read_sql_query(q, con)
high_batters = (
    high_batting_avgs.groupby("playerID")
    .agg(at_bat=("at_bat", "sum"), hits=("hits", "sum"))
    .assign(batting_average=lambda x: x.hits / x.at_bat)
)
high_batters.sort_values("batting_average", ascending=False).head().reset_index()

,playerID,at_bat,hits,batting_average
0,hazlebo01,134,54,0.402985
1,daviscu01,105,40,0.380952
2,fishesh01,254,95,0.374016
3,woltery01,138,51,0.369565
4,cobbty01,11436,4189,0.366299


In [28]:
q = """--sql
    SELECT * FROM teams
    """

mlb_players = pd.read_sql_query(q, con)
mlb_players

,ID,yearID,lgID,teamID,franchID,divID,div_ID,teamRank,G,Ghome,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1,1871,NA,BS1,BNA,None,NaN,3,31,NaN,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,2,1871,NA,CH1,CNA,None,NaN,2,28,NaN,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,3,1871,NA,CL1,CFC,None,NaN,8,29,NaN,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,4,1871,NA,FW1,KEK,None,NaN,7,19,NaN,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,5,1871,NA,NY2,NNA,None,NaN,5,33,NaN,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,2921,2019,NL,SLN,STL,C,6.0,1,162,81.0,...,168,0.989,St. Louis Cardinals,Busch Stadium III,3480393.0,98,97,STL,SLN,SLN
2921,2922,2019,AL,TBA,TBD,E,1.0,2,162,81.0,...,126,0.985,Tampa Bay Rays,Tropicana Field,1178735.0,97,96,TBR,TBA,TBA
2922,2923,2019,AL,TEX,TEX,W,2.0,3,162,81.0,...,143,0.982,Texas Rangers,Globe Life Park in Arlington,2132994.0,111,112,TEX,TEX,TEX
2923,2924,2019,AL,TOR,TOR,E,1.0,4,162,81.0,...,141,0.984,Toronto Blue Jays,Rogers Centre,1750114.0,97,98,TOR,TOR,TOR


In [29]:
q = """--sql
    SELECT salaries.yearID AS `Year`, 
           salaries.teamID AS `Team ID`,
           teams.name AS `Team Name`,
           salaries.playerID AS `Player ID`, 
           people.nameGiven AS `Player Name`,
           fielding.POS as `Position`,
           salaries.salary AS `Salary`
    FROM salaries
    JOIN fielding
        USING (yearID, teamID, playerID)
    INNER JOIN people on salaries.playerID=people.playerID
    INNER JOIN teams on salaries.teamID=teams.teamID
    """

teams_players_pos = pd.read_sql_query(q, con)
teams_players_pos

,Year,Team ID,Team Name,Player ID,Player Name,Position,Salary
0,1985,ATL,Atlanta Braves,barkele01,Leonard Harold,P,870000.0
1,1985,ATL,Atlanta Braves,barkele01,Leonard Harold,P,870000.0
2,1985,ATL,Atlanta Braves,barkele01,Leonard Harold,P,870000.0
3,1985,ATL,Atlanta Braves,barkele01,Leonard Harold,P,870000.0
4,1985,ATL,Atlanta Braves,barkele01,Leonard Harold,P,870000.0
...,...,...,...,...,...,...,...
2493621,2016,WAS,Washington Senators,zimmery01,Ryan Wallace,1B,14000000.0
2493622,2016,WAS,Washington Senators,zimmery01,Ryan Wallace,1B,14000000.0
2493623,2016,WAS,Washington Senators,zimmery01,Ryan Wallace,1B,14000000.0
2493624,2016,WAS,Washington Senators,zimmery01,Ryan Wallace,1B,14000000.0


In [30]:
q = """--sql
    SELECT salaries.yearID AS `Year`,
           salaries.teamID AS `Team`,
           salaries.playerID AS `Player ID`,
           people.nameGiven AS `Player Name`,
           fielding.POS as `Position`,
           salaries.salary AS `Salary`
    FROM salaries
    JOIN fielding
        USING (yearID, teamID, playerID)
    INNER JOIN people on salaries.playerID=people.playerID
    WHERE salaries.teamID IN ('BOS', 'NYA')
    """

mlb_players = pd.read_sql_query(q, con)
mlb_players["Team"] = mlb_players["Team"].replace(
    ["BOS", "NYA"], ["Boston Red Sox", "New York Yankees"]
)
mlb_players

,Year,Team,Player ID,Player Name,Position,Salary
0,1985,Boston Red Sox,armasto01,Antonio Rafael,OF,915000.0
1,1985,Boston Red Sox,barrema02,Martin Glenn,2B,272500.0
2,1985,Boston Red Sox,boggswa01,Wade Anthony,3B,1000000.0
3,1985,Boston Red Sox,boydoi01,Dennis Ray,P,177500.0
4,1985,Boston Red Sox,bucknbi01,William Joseph,1B,747500.0
...,...,...,...,...,...,...
2336,2016,New York Yankees,torrero01,Ronald Alcides,3B,508600.0
2337,2016,New York Yankees,torrero01,Ronald Alcides,OF,508600.0
2338,2016,New York Yankees,torrero01,Ronald Alcides,SS,508600.0
2339,2016,New York Yankees,willima07,Mason,OF,509700.0


In [31]:
avg_player_salary = (
    mlb_players.groupby(["Year", "Team"]).agg(avg_salary=("Salary", np.mean))
).reset_index()
avg_player_salary["avg_salary"] = avg_player_salary["avg_salary"].map(
    lambda x: float(f"{x:.2f}")
)
avg_player_salary

,Year,Team,avg_salary
0,1985,Boston Red Sox,397640.90
1,1985,New York Yankees,665985.90
2,1986,Boston Red Sox,476053.75
3,1986,New York Yankees,653034.33
4,1987,Boston Red Sox,677583.35
...,...,...,...
59,2014,New York Yankees,7637700.23
60,2015,Boston Red Sox,5270895.00
61,2015,New York Yankees,6878417.60
62,2016,Boston Red Sox,5628610.62


In [32]:
fig = px.bar(
    avg_player_salary,
    x="Year",
    y="avg_salary",
    color="Team",
    title="Player Salary | 1985-2016",
    barmode="group",
    text_auto="$.2s",
    color_discrete_map={"Boston Red Sox": "#BD3039", "New York Yankees": "#003087"},
)

fig.update_xaxes(insiderange=[1984.5, 2016.5])

fig.update_yaxes(title="Average Salary", tickprefix="$")

fig.update_layout(
    legend=dict(
        title="",
        orientation="h",
        yanchor="top",
        y=1.125,
        xanchor="left",
        x=0,
    )
)

fig.show()

In [33]:
avg_player_salary_by_pos = (
    mlb_players.groupby(["Team", "Position"]).agg(avg_salary=("Salary", np.mean))
).reset_index()

avg_player_salary_by_pos["avg_salary"] = avg_player_salary_by_pos["avg_salary"].map(
    lambda x: float(f"{x:.2f}")
)
avg_player_salary_by_pos

,Team,Position,avg_salary
0,Boston Red Sox,1B,3417392.71
1,Boston Red Sox,2B,1764983.05
2,Boston Red Sox,3B,2267240.70
3,Boston Red Sox,C,2023300.81
4,Boston Red Sox,OF,3265647.96
5,Boston Red Sox,P,2653076.31
6,Boston Red Sox,SS,1865799.92
7,New York Yankees,1B,5353727.09
8,New York Yankees,2B,2025167.55
9,New York Yankees,3B,4348392.01


In [34]:
fig = px.bar(
    avg_player_salary_by_pos,
    x="Position",
    y="avg_salary",
    color="Team",
    title="Player Salary by Position",
    text_auto="$.2s",
    barmode="group",
    color_discrete_map={"Boston Red Sox": "#BD3039", "New York Yankees": "#003087"},
)

fig.update_yaxes(title="Average Salary", tickprefix="$")

fig.update_layout(
    legend=dict(
        title="",
        orientation="h",
        yanchor="top",
        y=1.125,
        xanchor="left",
        x=0,
    )
)

fig.show()